# Objetivos deste trabalho:
- Se familiarizar com o ambiente Notebook e com Python
- Implementar um perceptron simples, treiná-lo no conjunto de TREINO do CIFAR-10 e avaliá-lo no conjunto de TESTE (alvo: distinguir fotos de animais de meios de transporte)
- Utilizar a função sigmóide e verificar seu efeito no treinamento e na avaliação
- Modificar a metodologia para classificar cada classe individualmente (i.e. treinar 10 perceptrons, um para cada classe). Considerar: dado um exemplo, que passará por cada perceptron, como decidir qual é a classe dele?

In [ ]:
%matplotlib inline

import torch
import torchvision
import numpy as np

np.seterr(all='raise')

In [ ]:
# Carregar os datasets

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print(len(dataset_train), len(dataset_test))

In [ ]:
# Converter para tons de cinza

# Treino
grayscale_dataset_train = []
for img,label in dataset_train:
    category = classes[label]
    gray_npimg = np.array(img.convert('L'))
    grayscale_dataset_train.append((gray_npimg,category))
    
# Teste
grayscale_dataset_test = []
for img,label in dataset_test:
    category = classes[label]
    gray_npimg = np.array(img.convert('L'))
    grayscale_dataset_test.append((gray_npimg,category))    

In [ ]:
# Mostrar uma imagem

import matplotlib.pyplot as plt
import numpy as np

image_index = 1
label = grayscale_dataset_train[image_index][1]
npimg = grayscale_dataset_train[image_index][0]

plt.imshow(npimg, cmap='gray')
plt.title(label)
plt.show()

In [ ]:
# Converter para vetores 1D

# Para pensar: por que a divisão por 255 no código abaixo?

# A divisão por 255 é feita para normalizar os tons de cinza entre 0 e 1

linear_dataset_train = []
target_labels = ('plane', 'car', 'ship', 'truck')
for img,category in grayscale_dataset_train:
    linear_img = img.reshape(img.shape[0]*img.shape[1],1) / 255
    if category in target_labels:
        label = 1
    else:
        label = 0
    linear_dataset_train.append((linear_img,label))
    
linear_dataset_test = []
target_labels = ('plane', 'car', 'ship', 'truck')
for img,category in grayscale_dataset_test:
    linear_img = img.reshape(img.shape[0]*img.shape[1],1) / 255
    if category in target_labels:
        label = 1
    else:
        label = 0
    linear_dataset_test.append((linear_img,label))    

In [ ]:
size = len(linear_dataset_train[0][0])
print(size)

# Definindo o perceptron

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def perceptron(inputs, weights):
    
    # Definição do somatório dos produtos das entradas com os pesos 
    o = np.sum(inputs * weights)
    
    # Retorna o resultado do somatório aplicado à função sigmoid
    y = sigmoid(o)
    
    return y

# Treinando o perceptron

In [ ]:
# Função avaliação
def evaluate(weights,dataset):
    # true positives, true negatives, false positives, false negatives
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for img, label in dataset:

        # Adiciona o bias nas entradas
        img = np.append(img, 1)
        
        # Pega a predição do perceptron
        y = perceptron(img, weights)
        
        # Ativação em 0.5
        if(y < 0.5):
            y = 0
        else:
            y = 1
        
        # Se a predição é igual ao valor real
        if y == label:
            # Então testa se é tp ou tn
            if y == 1:
                tp += 1
            else:
                tn += 1
        # Senão testa se é fn ou fp
        else:
            if y == 0:
                fn += 1
            else:
                fp += 1

    # Acurácia
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    
    # Sensitividade
    recall = tp/(tp+fn)
    
    # Especificidade
    specificity = tn/(tn+fp)
    
    # Retorna as métricas
    return accuracy, recall, specificity
    
# Função de atualização dos pesos do perceptron
def update_weights(inputs, weights, y, t, neta):
    return weights + neta * (t-y) * inputs * (y * (1-y))

In [ ]:
# Inicialização
weights = (np.random.rand(1,size) - 0.5)[0]
bias = (np.random.rand(1) - 0.5) 
weights = np.append(weights, bias)

neta = 0.0001

# Implemente o treino aqui (para separar as duas classes definidas)
accuracies = []

for epoch in range(100):
    np.random.shuffle(linear_dataset_train)
    
    # Para cada exemplo do dataset
    for example, target in linear_dataset_train:
        
        # Adiciona o bias nas entradas do exemplo
        example = np.append(example, 1)
        
        # Passa as entradas do exemplo e os pesos para o perceptron
        y = perceptron(example, weights)
        
        # Atualiza os pesos
        weights = update_weights(example, weights, y, target, neta)
    
    # Calcula as métricas de avaliação para a época
    accuracy, recall, specificity = evaluate(weights,linear_dataset_train)
    # Exibe-as na tela
    print('Época:', epoch, 'Acurácia:', accuracy, 'Sensitividade', recall, 'Especificidade:', specificity)

In [ ]:
# Avalie o modelo treinado aqui
# Como a acurácia no conjunto de teste se compara com a acurácia obtida no conjunto de treino?



In [ ]:
# Caso queiram plotar alguma coisa

import matplotlib.pyplot as plt
plt.plot(accuracies)

# Classificando classes individuais

Implemente aqui a modificação do processo de avaliação e treinamento para poder classificar cada classe individualmente.

- Ideia geral: treinar um perceptron por classe (exemplo positivo = exemplos da classe; exemplos negativos = exemplo de todas outras classes)
- Dado um exemplo qualquer, como decidir qual perceptron está dando a classe correta?